In [1]:
import pandas as pd , requests
import base64, json , os, uuid
import re, librosa

In [2]:
def extract_student_data(xl_file,school):
    student_names = []
    classes = []
    sections = []
    student_roll_nos = []
    student_ids = []
    for row in range(len(xl_file)):
        values = xl_file.iloc[row]
        # print(values)
        name = values["firstname"] + values["lastname"]
        Class = values["class"]
        section = values["section"]
        roll_no = values["rollnumber"]
        random_uuid = uuid.uuid4()
        # print(random_uuid)
        student_ids.append(random_uuid)
        student_names.append(name)
        classes.append(Class)
        sections.append(section)
        student_roll_nos.append(roll_no)
    data = {
        "student_id" : student_ids,
        "name":student_names,
        "class":classes,
        "section":sections,
        "roll_no":student_roll_nos,
        "school":[school]*len(xl_file)
    }
    df = pd.DataFrame(data)
    # Created unique ids for students of Class 1A
    df.to_csv(f"{Class}{section}_student_data.csv",index=False)

In [2]:
def generate_audio_report(xl_file,audios_folder,rca):
    audios = os.listdir(audios_folder)
    audio_exist = {}
    for audio in audios:
        audio_roll_no = audio.replace(".ogg","").replace(".mp3","")[::-1].split()[0][::-1]
        audio_exist[audio_roll_no] = f"{audios_folder}/{audio}"
    print(audio_exist)
    student_ids = []
    RCA = [] # Integer
    partA_scores = []
    partB_scores = []
    totals = []
    grades = []
    partA_reports = [] # dump the json here
    partB_reports = []
    feedbacks = []
    durations = []
    for row in range(len(xl_file)):
        id, name, Class, section, roll_no, school = xl_file.iloc[row].values
        if str(roll_no).strip() not in audio_exist:
            print("Audio not exist for", roll_no)
        else:
            audio_file_path = audio_exist[str(roll_no).strip()]
            audio_data, sample_rate = librosa.load(audio_file_path, sr=None)
            duration_in_seconds = librosa.get_duration(y=audio_data, sr=sample_rate)
            print(roll_no, audio_file_path)
            with open(audio_file_path, "rb") as file:
                file_data = bytearray(file.read())

            file_base64 = base64.b64encode(file_data).decode('utf-8')
            print(len(file_base64))
            body = {
                "studentClass":int(Class),
                "audioBufferBase64":file_base64,
                "rca":rca
            }
            report_response = requests.post("http://localhost:5000/api/get-report",json=body)
            report_json = report_response.json()
            partA_score = report_json["partA_score"]["score"]
            feedback = report_json["feedback"]
            # id, RCA, partA_score, partB_score, total, grade, partA_report, feedback
            student_ids.append(id)
            RCA.append(rca)
            partA_scores.append(partA_score)
            partB_scores.append(" ")
            partB_reports.append(" ")
            totals.append(" ")
            grades.append(" ")
            partA_reports.append(json.dumps(report_json))
            feedbacks.append(json.dumps(feedback))
            durations.append(duration_in_seconds)
            

    student_reports_data = {
        "student_id" : student_ids,
        "rca" : RCA,
        "partA_score":partA_scores,
        "partB_score":partB_scores,
        "total":totals,
        "grade":grades,
        "partA_report":partA_reports,
        "partB_report":partB_reports,
        "feedback":feedbacks,
        "duration":durations
    }
    report_df = pd.DataFrame(student_reports_data)
    report_df.to_csv(f"{Class}{section}_student_audio_report.csv",index=False)

In [3]:
def re_generate_audio_report(student_data,student_audio_report,audios_folder,rca):
    ids_in_reportfile = student_audio_report["student_id"].values
    ids_in_reportfile = ids_in_reportfile.tolist()
    audios = os.listdir(audios_folder)
    audio_exist = {}
    for audio in audios:
        audio_roll_no = audio.replace(".ogg","").replace(".mp3","")[::-1].split()[0][::-1]
        audio_exist[audio_roll_no] = f"{audios_folder}/{audio}"
    student_ids = []
    RCA = [] # Integer
    partA_scores = []
    partB_scores = []
    totals = []
    grades = []
    partA_reports = [] # dump the json here
    partB_reports = []
    durations = []
    for row in range(len(student_data)):
        data = student_data.iloc[row]
        Class = data["class"]
        section = data["section"]
        roll_no = data["roll_no"]
        if data["student_id"] not in ids_in_reportfile and str(data["roll_no"]) in audio_exist:
            audio_file_path = audio_exist[str(roll_no).strip()]
            try:
                audio_data, sample_rate = librosa.load(audio_file_path, sr=None,dtype=float)
                duration_in_seconds = librosa.get_duration(y=audio_data, sr=sample_rate)
            except:
                print("Couldn' read duration for",roll_no,audio_file_path)
            print(roll_no, audio_file_path)
            with open(audio_file_path, "rb") as file:
                file_data = bytearray(file.read())

            file_base64 = base64.b64encode(file_data).decode('utf-8')
            print(len(file_base64))
            body = {
                "studentClass":int(Class),
                "audioBufferBase64":file_base64,
                "rca":rca
            }
            report_response = requests.post("http://localhost:5000/api/get-report",json=body)
            report_json = report_response.json()
            partA_score = report_json["partA_score"]["score"]
            feedback = report_json["feedback"]
            # id, RCA, partA_score, partB_score, total, grade, partA_report, feedback
            student_ids.append(data["student_id"])
            RCA.append(1)
            partA_scores.append(partA_score)
            partB_scores.append(" ")
            partB_reports.append(" ")
            totals.append(" ")
            grades.append(" ")
            partA_reports.append(json.dumps(report_json))
            durations.append(duration_in_seconds)
            
        elif data["student_id"] in ids_in_reportfile:
            audio_file_path = audio_exist[str(roll_no).strip()]
            try:
                audio_data, sample_rate = librosa.load(audio_file_path, sr=None,dtype=float)
                duration_in_seconds = librosa.get_duration(y=audio_data, sr=sample_rate)
            except:
                print("Couldn' read duration for",roll_no,audio_file_path)
            audio_report_data = student_audio_report.iloc[ids_in_reportfile.index(data["student_id"])]
            student_ids.append(audio_report_data["student_id"])
            RCA.append(audio_report_data["rca"]) # Integer
            partA_scores.append(audio_report_data["partA_score"])
            partB_scores.append(audio_report_data["partB_score"])
            totals.append(audio_report_data["total"])
            grades.append(audio_report_data["grade"])
            partA_reports.append(audio_report_data["partA_report"]) # dump the json here
            partB_reports.append(audio_report_data["partB_report"])
            durations.append(duration_in_seconds)
    student_reports_data = {
        "student_id" : student_ids,
        "rca" : RCA,
        "partA_score":partA_scores,
        "partB_score":partB_scores,
        "total":totals,
        "grade":grades,
        "partA_report":partA_reports,
        "partB_report":partB_reports,
        "duration":durations
    }
    report_df = pd.DataFrame(student_reports_data)
    report_df.to_csv(f"{Class}{section}_student_audio_report.csv",index=False)

In [4]:
def final_report(partb_xl_file,student_report_file,student_data_file):
    partB_responses = {}
    part_b_roll_no = {}
    for row in range(len(partb_xl_file)):
        values = partb_xl_file.iloc[row]
        Class = values["class"]
        section = values["section"]
        roll_no = values["rollnumber"]
        part_b_roll_no[str(roll_no)] = 1
        response = [res.strip() for res in re.findall(r'\[([^\]]+)\]' , values["Response 2"] ) ]
        right_answer = [res.strip() for res in re.findall(r'\[([^\]]+)\]' , values["Right answer 2"] ) ]
        marks = 0
        for answer in range(min(len(response),len(right_answer))):
            if response[answer] == right_answer[answer]:
                marks += 10
        partB_responses[str(roll_no)] = {
            "marks":marks,
            "partB_report" :response
        }
    roll_no_mapped_student_id = {}
    for row in range(len(student_data_file)):
        data_values = student_data_file.iloc[row]
        roll_no_mapped_student_id[data_values["student_id"]] = str(data_values["roll_no"])
    
    student_ids = [] 
    RCA = [] 
    partA_scores = []
    partB_scores = [] 
    totals = []
    grades = []
    partA_reports = []
    partB_reports = []
    durations = []
    for row in range(len(student_report_file)):
        values = student_report_file.iloc[row]
        if roll_no_mapped_student_id[values["student_id"]] in part_b_roll_no :
            student_ids.append(values["student_id"])
            RCA.append(values["rca"])
            partA_scores.append(values["partA_score"])
            partB_scores.append(partB_responses[roll_no_mapped_student_id[values["student_id"]]]["marks"]*2)
            total = (values["partA_score"] + partB_responses[roll_no_mapped_student_id[values["student_id"]]]["marks"]*2)/2
            totals.append(total)
            if total<50:
                grades.append("Emergent")
            elif total<80:
                grades.append("Transitional")
            else:
                grades.append("Proficient")
            partA_reports.append(values["partA_report"])
            partB_reports.append('@'.join(partB_responses[roll_no_mapped_student_id[values["student_id"]]]["partB_report"]))
            durations.append(values["duration"])
    data = {
        "student_id" : student_ids,
        "rca" : RCA,
        "partA_score":partA_scores,
        "partB_score":partB_scores,
        "total":totals,
        "grade":grades,
        "partA_report":partA_reports,
        "partB_report":partB_reports,
        "duration":durations
    }

    final_report = pd.DataFrame(data)
    final_report.to_csv(f"{Class}{section}_student_final_report.csv",index=False)

Class 1 A

In [8]:
class_1a_partb = pd.read_excel("part b\Class - 1A.xlsx")
class_1a_student_data = pd.read_csv("student_data/1A_student_data.csv")
class_1a_audio_report = pd.read_csv("1A_student_audio_report.csv")
final_report(partb_xl_file=class_1a_partb,student_data_file=class_1a_student_data,student_report_file=class_1a_audio_report)

Class 1B

In [11]:
class_1b_partb = pd.read_excel("part b\Class - 1B.xlsx")
class_1b_parta_report = pd.read_csv("1B_student_audio_report.csv")
class_1b_student_data = pd.read_csv("student_data/1B_student_data.csv")
final_report(partb_xl_file=class_1b_partb,student_report_file=class_1b_parta_report,student_data_file=class_1b_student_data)

Class 2A

In [13]:
class_2a_partb = pd.read_excel("part b\Class - 2A.xlsx")
class_2a_audio_report = pd.read_csv("2A_student_audio_report.csv")
class_2a_student_data = pd.read_csv("student_data/2A_student_data.csv")
final_report(partb_xl_file=class_2a_partb,student_report_file=class_2a_audio_report,student_data_file=class_2a_student_data)

Class 2B

In [14]:
class_2b_partb = pd.read_excel("part b\Class-2B.xlsx")
class_2b_audio_report = pd.read_csv("2B_student_audio_report.csv")
class_2b_student_data = pd.read_csv("student_data/2B_student_data.csv")
final_report(partb_xl_file=class_2b_partb,student_report_file=class_2b_audio_report,student_data_file=class_2b_student_data)

Class 3A

In [15]:
class_3a_partb = pd.read_excel("part b\Class -3A.xlsx")
class_3a_parta_report = pd.read_csv("3A_student_audio_report.csv")
class_3a_student_data = pd.read_csv("student_data/3A_student_data.csv")
final_report(partb_xl_file=class_3a_partb,student_data_file=class_3a_student_data,student_report_file=class_3a_parta_report)

Class 3B

In [16]:
class_3b_partb = pd.read_excel("part b\Class - 3B.xlsx")
class_3b_audio_report = pd.read_csv("3B_student_audio_report.csv")
class_3b_student_data = pd.read_csv("student_data/3B_student_data.csv")
final_report(partb_xl_file=class_3b_partb,student_report_file=class_3b_audio_report,student_data_file=class_3b_student_data)

Class 4A

In [17]:
class_4a_partb = pd.read_excel("part b\Class - 4A.xlsx")
class_4a_student_data = pd.read_csv("student_data/4A_student_data.csv")
class_4a_parta_report = pd.read_csv("4A_student_audio_report.csv")
final_report(partb_xl_file=class_4a_partb,student_data_file=class_4a_student_data,student_report_file=class_4a_parta_report)

Class 4B

In [18]:
class_4b_student_data = pd.read_csv("student_data/4B_student_data.csv")
class_4b_audio_report = pd.read_csv("4B_student_audio_report.csv")
class_4b_partb = pd.read_excel("part b\Class - 4B.xlsx")
final_report(partb_xl_file=class_4b_partb,student_report_file=class_4b_audio_report,student_data_file=class_4b_student_data)

Class 5A

In [19]:
class_5a_partb = pd.read_excel("part b\Class - 5A.xlsx")
class_5a_audio_report = pd.read_csv("5A_student_audio_report.csv")
class_5a_student_data = pd.read_csv("student_data/5A_student_data.csv")
final_report(partb_xl_file=class_5a_partb,student_report_file=class_5a_audio_report,student_data_file=class_5a_student_data)

Class 5B

In [20]:
class_5b_partb = pd.read_excel("part b\Class - 5B.xlsx")
class_5b_audio_report  = pd.read_csv("5B_student_audio_report.csv")
class_5b_student_data = pd.read_csv("student_data/5B_student_data.csv")
final_report(partb_xl_file=class_5b_partb,student_report_file=class_5b_audio_report,student_data_file=class_5b_student_data)